In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn

## Competition Utility Function

In [404]:
#!/usr/bin/env python3

# This file contains functions for computing scores for the 2019 PhysioNet/CinC
# challenge.
#
# Written by M. Reyna on 1 February 2019.  Last updated on 18 February 2019.
#
# The compute_scores_2019 function computes a normalized utility score for a
# cohort of patients as well as several traditional scoring metrics.
#
# Inputs:
#   'labels_directory' is a directory of pipe-delimited text files containing a
#   binary vector of labels indicating whether a patient is not septic (0) or
#   septic (1).
#
#   'predictions_directory' is a directory of pipe-delimited text files, where
#   the first column of the file gives the  predicted probability that the
#   patient is septic at each time, and the second column of the file is a
#   binarized version of this vector. Note that there must be a prediction for
#   every label.
#
#   Note: Currently, filenames in labels_directory and predictions_directory
#   must be the same, and the extension must be .psv.
#
# Output:
#   'output_file' is a pipe-delimited text file (optional) that gives AUROC,
#   AUPRC, accuracy, F-measure, and utility scores for a cohort of patients.
#
# Example:
#
#   In [1]: compute_scores_2019('labels', 'predictions')
#   Out[1]: auroc, auprc, accuracy, f_measure, utility

import numpy as np, os, os.path, sys, argparse
from collections import defaultdict

def compute_scores_2019(label_directory, prediction_directory):
    # Set parameters.
    label_header       = 'SepsisLabel'
    prediction_header  = 'PredictedLabel'
    probability_header = 'PredictedProbability'

    dt_early   = -12
    dt_optimal = -6
    dt_late    = 3

    max_u_tp = 1
    min_u_fn = -2
    u_fp     = -0.05
    u_tn     = 0

    # Find label and prediction files.
    label_files = []
    for filename in os.listdir(label_directory):
        full_filename = os.path.join(label_directory, filename)
        if os.path.isfile(full_filename) and full_filename.endswith(''):
            label_files.append(filename)
    label_files = sorted(label_files)

    prediction_files = []
    for filename in os.listdir(prediction_directory):
        full_filename = os.path.join(prediction_directory, filename)
        if os.path.isfile(full_filename) and full_filename.endswith(''):
            prediction_files.append(filename)
    prediction_files = sorted(prediction_files)

    if len(label_files) != len(prediction_files):
        raise Exception('Numbers of labels and predictions must be the same.')

    # Load labels and predictions.
    num_files            = len(label_files)
    cohort_labels        = []
    cohort_predictions   = []
    cohort_probabilities = []

    for k in range(num_files):
        labels        = load_column(os.path.join(label_directory, label_files[k]), label_header)
        predictions   = load_column(os.path.join(prediction_directory, prediction_files[k]), prediction_header)
        probabilities = load_column(os.path.join(prediction_directory, prediction_files[k]), probability_header)

        # Check labels and predictions for errors.
        if not (len(labels) == len(predictions) and len(predictions) == len(probabilities)):
            raise Exception('Numbers of labels and predictions must be the same.')

        num_records = len(labels)

        for i in range(num_records):
            if labels[i] not in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

            if predictions[i] not in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

            if not 0 <= probabilities[i] <= 1:
                raise Exception('Probabilities must satisfy 0 <= probability <= 1.')

        if 0<np.sum(predictions)<num_records:
            min_probability_positive = np.min(probabilities[predictions == 1])
            max_probability_negative = np.max(probabilities[predictions == 0])

            if min_probability_positive <= max_probability_negative:
                raise Exception('Predictions are inconsistent with probabilities, i.e., a positive prediction has a lower (or equal) probability than a negative prediction.')

        # Record labels and predictions.
        cohort_labels.append(labels)
        cohort_predictions.append(predictions)
        cohort_probabilities.append(probabilities)

    # Compute AUC, accuracy, and F-measure.
    labels        = np.concatenate(cohort_labels)
    predictions   = np.concatenate(cohort_predictions)
    probabilities = np.concatenate(cohort_probabilities)

    auroc, auprc        = compute_auc(labels, probabilities)
    accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

    # Compute utility.
    observed_utilities = np.zeros(num_files)
    best_utilities     = np.zeros(num_files)
    worst_utilities    = np.zeros(num_files)
    inaction_utilities = np.zeros(num_files)

    for k in range(num_files):
        labels = cohort_labels[k]
        num_records          = len(labels)
        observed_predictions = cohort_predictions[k]
        best_predictions     = np.zeros(num_records)
        worst_predictions    = np.zeros(num_records)
        inaction_predictions = np.zeros(num_records)

        if any(labels):
            t_sepsis = min(i for i, label in enumerate(labels) if label) - dt_optimal
            best_predictions[max(0, t_sepsis + dt_early) : min(t_sepsis + dt_late, num_records)] = 1
        else:
            best_predictions[:] = 0
        worst_predictions = 1 - best_predictions

        observed_utilities[k] = compute_prediction_utility(labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        best_utilities[k]     = compute_prediction_utility(labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        worst_utilities[k]    = compute_prediction_utility(labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        inaction_utilities[k] = compute_prediction_utility(labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

    unnormalized_observed_utility = np.sum(observed_utilities)
    unnormalized_best_utility     = np.sum(best_utilities)
    unnormalized_worst_utility    = np.sum(worst_utilities)
    unnormalized_inaction_utility = np.sum(inaction_utilities)

    if not (unnormalized_worst_utility <= unnormalized_best_utility and unnormalized_inaction_utility <= unnormalized_best_utility):
        raise Exception('Optimal utility must be higher than inaction utility.')

    normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (unnormalized_best_utility - unnormalized_inaction_utility)

    return auroc, auprc, accuracy, f_measure, normalized_observed_utility

# The load_column function loads a column from a table.
#
# Inputs:
#   'filename' is a string containing a filename.
#
#   'header' is a string containing a header.
#
# Outputs:
#   'column' is a vector containing a column from the file with the given
#   header.
#
# Example:
#
#   Omitted.

def load_column(filename, *headers):
    header_to_index = defaultdict(list)
    header_to_column = defaultdict(list)
    with open(filename, 'r') as f:
        for i, l in enumerate(f):
            arrs = l.strip().split('|')
            if i == 0:
                for header in headers:
                    try:
                        header_to_index[header] = arrs.index(header)
                    except:
                        raise Exception('{} must contain column with header {} containing numerical entries.'.format(filename, header))
            else:
                for header in headers:
                    try:
                        header_to_column[header].append(float(arrs[header_to_index[header]]))
                    except:
                        raise Exception('{} must contain column with header {} containing numerical entries.'.format(filename, header))
    columns = [np.array(header_to_column[header]) for header in headers]

    if len(headers) == 1:
        return columns[0]
    else:
        return columns

# The compute_auc function computes AUROC and AUPRC as well as other summary
# statistics (TP, FP, FN, TN, TPR, TNR, PPV, NPV, etc.) that can be exposed
# from this function.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a probability vector, where predictions[i] gives the
#   predicted probability that the patient is septic at time i.  Note that there
#   must be a prediction for every label, i.e, len(labels) ==
#   len(predictions).
#
# Outputs:
#   'auroc' is a scalar that gives the AUROC of the classifier using its
#   predicted probabilities, where specificity is interpolated for intermediate
#   sensitivity values.
#
#   'auprc' is a scalar that gives the AUPRC of the classifier using its
#   predicted probabilities, where precision is a piecewise constant function of
#   recall.
#
# Example:
#
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0.3, 0.4, 0.6, 0.7, 0.8, 0.8]
#   In [3]: auroc, auprc = compute_auc(labels, predictions)
#   In [4]: auroc
#   Out[4]: 1.0
#   In [5]: auprc
#   Out[5]: 1.0

def compute_auc(labels, predictions):
    # Check inputs for errors.
    if len(predictions) != len(labels):
        raise Exception('Numbers of predictions and labels must be the same.')

    n = len(labels)
    for i in range(n):
        if not labels[i] in (0, 1):
            raise Exception('Labels must satisfy label == 0 or label == 1.')

    for i in range(n):
        if not 0 <= predictions[i] <= 1:
            raise Exception('Predictions must satisfy 0 <= prediction <= 1.')

    # Find prediction thresholds.
    thresholds = np.unique(predictions)[::-1]
    if thresholds[0] != 1:
        thresholds = np.concatenate((np.array([1]), thresholds))

    if thresholds[-1] != 0:
        thresholds = np.concatenate((thresholds, np.array([0])))
    m = len(thresholds)

    # Populate contingency table across prediction thresholds.
    tp = np.zeros(m)
    fp = np.zeros(m)
    fn = np.zeros(m)
    tn = np.zeros(m)

    # Find indices that sort predicted probabilities from largest to smallest.
    idx = np.argsort(predictions)[::-1]

    i = 0
    for j in range(m):
        # Initialize contingency table for j-th prediction threshold.
        if j == 0:
            tp[j] = 0
            fp[j] = 0
            fn[j] = np.sum(labels == 1)
            tn[j] = np.sum(labels == 0)
        else:
            tp[j] = tp[j - 1]
            fp[j] = fp[j - 1]
            fn[j] = fn[j - 1]
            tn[j] = tn[j - 1]

        # Update contingency table for i-th largest prediction probability.
        while i < n and predictions[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Summarize contingency table.
    tpr = np.zeros(m)
    tnr = np.zeros(m)
    ppv = np.zeros(m)
    npv = np.zeros(m)

    for j in range(m):
        if tp[j] + fn[j]:
            tpr[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr[j] = 1
        if fp[j] + tn[j]:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = 1
        if tp[j] + fp[j]:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = 1
        if fn[j] + tn[j]:
            npv[j] = tn[j] / (fn[j] + tn[j])
        else:
            npv[j] = 1

    # Compute AUROC as the area under a piecewise linear function of TPR /
    # sensitivity (x-axis) and TNR / specificity (y-axis) and AUPRC as the area
    # under a piecewise constant of TPR / recall (x-axis) and PPV / precision
    # (y-axis).
    auroc = 0
    auprc = 0
    for j in range(m-1):
        auroc += 0.5 * (tpr[j + 1] - tpr[j]) * (tnr[j + 1] + tnr[j])
        auprc += (tpr[j + 1] - tpr[j]) * ppv[j + 1]

    return auroc, auprc

# The compute_accuracy_f_measure function computes the accuracy and F-measure
# for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'accuracy' is a scalar that gives the accuracy of the classifier using its
#   binarized predictions.
#
#   'f_measure' is a scalar that gives the F-measure of the classifier using its
#   binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: accuracy, f_measure = compute_prediction_utility(labels, predictions)
#   In [4]: accuracy
#   Out[4]: 0.666666666667
#   In [5]: f_measure
#   Out[5]: 0.666666666667

def compute_accuracy_f_measure(labels, predictions):
    # Check inputs for errors.
    if len(predictions) != len(labels):
        raise Exception('Numbers of predictions and labels must be the same.')

    n = len(labels)
    for i in range(n):
        if not labels[i] in (0, 1):
            raise Exception('Labels must satisfy label == 0 or label == 1.')

    for i in range(n):
        if not predictions[i] in (0, 1):
            raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

    # Populate contingency table.
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(n):
        if labels[i] and predictions[i]:
            tp += 1
        elif labels[i] and not predictions[i]:
            fp += 1
        elif not labels[i] and predictions[i]:
            fn += 1
        elif not labels[i] and not predictions[i]:
            tn += 1

    # Summarize contingency table.
    if tp + fp + fn + tn:
        accuracy = float(tp + tn) / float(tp + fp + fn + tn)
    else:
        accuracy = 1.0

    if 2 * tp + fp + fn:
        f_measure = float(2 * tp) / float(2 * tp + fp + fn)
    else:
        f_measure = 1.0

    return accuracy, f_measure

# The compute_prediction_utility function computes the total time-dependent
# utility for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'utility' is a scalar that gives the total time-dependent utility of the
#   classifier using its binarized predictions.
#
# Example:
#   In [1]: labels = [0 0 0 0 1 1]
#   In [2]: predictions = [0 0 1 1 1 1]
#   In [3]: utility = compute_prediction_utility(labels, predictions)
#   In [4]: utility
#   Out[4]: 0.444444444444

def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0):
    # Check inputs for errors.
    if len(predictions) != len(labels):
        raise Exception('Numbers of predictions and labels must be the same.')

    n = len(labels)
    for i in range(n):
        if not labels[i] in (0, 1):
            raise Exception('Labels must satisfy label == 0 or label == 1.')

    for i in range(n):
        if not predictions[i] in (0, 1):
            raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

    if dt_early >= dt_optimal:
        raise Exception('The earliest beneficial time for predictions must be before the optimal time.')

    if dt_optimal >= dt_late:
        raise Exception('The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if any(labels):
        is_septic = True
        t_sepsis = min(i for i, label in enumerate(labels) if label) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    # Define slopes and intercept points for affine utility functions of the
    # form u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)


## Read imputed files and creating training matrix using windows

In [337]:
def create_train_data(window_size=6, impute_method='mean'):
    X_train = []
    y_train = []
    features = None

    i = 0
    for file in os.listdir('../data/train_%s_imputed/'%impute_method):

        # Read file
        patient_df = []
        with open('../data/train_%s_imputed/%s' % (impute_method, file)) as f:

            if not features:
                features = f.readline().rstrip('\n').split('|')
            else:
                # This skips the headers
                f.readline()

            for idx, line in enumerate(f):
                line = line.rstrip('\n')
                patient_df.append(line.split('|'))

        patient_df = np.array(patient_df)
        
        # Get sliding-window data
        window_start = 0
        while (window_start + window_size) <= patient_df.shape[0]:
            window_data = patient_df[window_start:window_start + window_size, 1:37].astype(np.float)
            assert window_data.shape[0] == window_size

            x_i = np.reshape(window_data, (window_size*window_data.shape[1],))
            X_train.append(x_i)

            label = int(patient_df[window_start + window_size - 1, 41])
            y_train.append(label)

            window_start += 1
    
    return np.array(X_train), np.array(y_train), features

In [338]:
X_train, y_train, features = create_train_data()

In [339]:
X_train.shape, y_train.shape

((130976, 216), (130976,))

In [388]:
features[1:37]

['HR',
 'O2Sat',
 'Temp',
 'SBP',
 'MAP',
 'DBP',
 'Resp',
 'EtCO2',
 'BaseExcess',
 'HCO3',
 'FiO2',
 'pH',
 'PaCO2',
 'SaO2',
 'AST',
 'BUN',
 'Alkalinephos',
 'Calcium',
 'Chloride',
 'Creatinine',
 'Bilirubin_direct',
 'Glucose',
 'Lactate',
 'Magnesium',
 'Phosphate',
 'Potassium',
 'Bilirubin_total',
 'TroponinI',
 'Hct',
 'Hgb',
 'PTT',
 'WBC',
 'Fibrinogen',
 'Platelets',
 'Age',
 'Gender']

In [341]:
y_train.sum()

1799

## Train Models and Predict

In [259]:
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, precision_score, recall_score

In [371]:
# Read test files, create windows, and predict. 
def predict(model, model_type, window_size=6, impute_method='mean'):
    X_test = []
    y_test = []
    features = None
    
    for file in os.listdir('../data/test_%s_imputed/'%(impute_method)):
        X = []
        y = []
        
        # Read file 
        patient_df = []
        with open('../data/test_%s_imputed/%s' % (impute_method, file)) as f:

            if not features:
                features = f.readline().rstrip('\n').split('|')
            else:
                # This skips the headers
                f.readline()

            for idx, line in enumerate(f):
                line = line.rstrip('\n') 
                patient_df.append(line.split('|'))
        
        # Create patient df
        patient_df = np.array(patient_df)
        X_patient = np.concatenate((patient_df[:,1:37], patient_df[:,41][:,None]), axis=1).astype(np.float)
        
        assert X_patient.shape[0] == patient_df.shape[0]
        
        # Append 5 rows of mean values to top of df 
        mean_vals = np.mean(X_patient, axis=0)
        X_patient = np.concatenate(([mean_vals]*max(1,window_size-1), X_patient))
    
        # Get data for every window
        window_start = 0
        while (window_start + window_size) <= X_patient.shape[0]:
            window_data = X_patient[window_start:window_start + window_size, 0:X_patient.shape[1]-1]
            assert window_data.shape[0] == window_size

            x_i = np.reshape(window_data, (window_size*window_data.shape[1],))
            X.append(x_i)
            
            label = int(X_patient[window_start+window_size-1, X_patient.shape[1]-1])
            y.append(label)

            window_start += 1
        
        # Add X_test to X
        X_test.extend(list(X))
        y_test.extend(list(y))
        
        # Predict for every time step
        y_pred = model.predict(X)
        y_pred_prob = model.predict_proba(X)
        
        assert y_pred.shape[0] == len(y)
        
        # Save to file
        if not os.path.isdir('../data/model_predictions/%s_pred_%s_imputed_%d'%(model_type, impute_method, window_size)):
            os.mkdir('../data/model_predictions/%s_pred_%s_imputed_%d'%(model_type, impute_method, window_size))
            
        pred = np.transpose(np.vstack((y_pred_prob[:,1], y_pred)))

        pred_df = pd.DataFrame(pred, columns=["PredictedProbability", "PredictedLabel"])
        pred_df.to_csv('../data/model_predictions/%s_pred_%s_imputed_%d/%s'%(model_type, impute_method, window_size, file), sep='|', index=False)
        
    return np.array(X_test), np.array(y_test)

In [406]:
auroc, auprc, accuracy, f_measure, utility = compute_scores_2019("../data/test_forw_imputed/", 
                                                                 '../data/model_predictions/RLR_pred_forw_imputed_3/')
auroc, auprc, accuracy, f_measure, utility 

(0.6930903100200748,
 0.03831537709057811,
 0.6847399738506283,
 0.05411896565527179,
 -0.003938115330520376)

## Regularized Log. Reg. 

In [374]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

lr_model = LogisticRegressionCV(cv=5, solver='lbfgs', max_iter=1000, class_weight='balanced')

# window size, imp method, train acc, train recall, train prec., test acc, test auc, 
# test recall, test prec.
 
lr_results = []

for ws in range(1,7):
    for imp in ['mean', 'forw']:
        res = [ws, imp]
        print("Working on ws: %d, imp: %s"%(ws,imp))
        
        X_train, y_train, _ = create_train_data(window_size=ws, impute_method=imp)
        lr_model = lr_model.fit(X_train, y_train)
        
        res.append(lr_model.score(X_train, y_train))
        y_pred = lr_model.predict(X_train)
        res.extend([recall_score(y_train, y_pred), precision_score(y_train, y_pred)])
        
        X_test, y_test = predict(lr_model, 'RLR', window_size=ws, impute_method=imp)
        res.append(lr_model.score(X_test, y_test))
        
        y_pred_prob = lr_model.predict_proba(X_test)
        res.append(roc_auc_score(y_test, y_pred_prob[:,1]))
        
        y_pred = lr_model.predict(X_test)
        res.extend([recall_score(y_test, y_pred), precision_score(y_test, y_pred)])
        
        results.append(res)

In [380]:
lr_results = np.array(results)

In [392]:
lr_results.tofile('../results/lr_results', sep='\n')

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(max_depth=5, class_weight='balanced')

rf_results = []

for ws in range(1,7):
    for imp in ['mean', 'forw']:
        res = [ws, imp]
        print("Working on ws: %d, imp: %s"%(ws,imp))
        
        X_train, y_train, _ = create_train_data(window_size=ws, impute_method=imp)
        rf_model = rf_model.fit(X_train, y_train)
        
        res.append(rf_model.score(X_train, y_train))
        y_pred = rf_model.predict(X_train)
        res.extend([recall_score(y_train, y_pred), precision_score(y_train, y_pred)])
        
        X_test, y_test = predict(rf_model, 'RF', window_size=ws, impute_method=imp)
        res.append(rf_model.score(X_test, y_test))
        
        y_pred_prob = rf_model.predict_proba(X_test)
        res.append(roc_auc_score(y_test, y_pred_prob[:,1]))
        
        y_pred = rf_model.predict(X_test)
        res.extend([recall_score(y_test, y_pred), precision_score(y_test, y_pred)])
        
        rf_results.append(res)

In [389]:
rf_results = np.array(rf_results)

In [396]:
rf_results.tofile('../results/rf_results', sep='\n')

## SVM

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(class_weight='balanced', max_iter=1000, probability=True)

svm_results = []

for ws in range(1,7):
    for imp in ['mean', 'forw']:
        res = [ws, imp]
        print("Working on ws: %d, imp: %s"%(ws,imp))
        
        X_train, y_train, _ = create_train_data(window_size=ws, impute_method=imp)
        svm_model = svm_model.fit(X_train, y_train)
        
        res.append(svm_model.score(X_train, y_train))
        y_pred = svm_model.predict(X_train)
        res.extend([recall_score(y_train, y_pred), precision_score(y_train, y_pred)])
        
        X_test, y_test = predict(svm_model, 'SVM', window_size=ws, impute_method=imp)
        res.append(svm_model.score(X_test, y_test))
        
        y_pred_prob = svm_model.predict_proba(X_test)
        res.append(roc_auc_score(y_test, y_pred_prob[:,1]))
        
        y_pred = svm_model.predict(X_test)
        res.extend([recall_score(y_test, y_pred), precision_score(y_test, y_pred)])
        
        svm_results.append(res)

## XG Boost

In [393]:
from sklearn.ensemble import GradientBoostingClassifier

xg_model = GradientBoostingClassifier(max_features='auto')

xg_results = []

for ws in range(1,7):
    for imp in ['mean', 'forw']:
        res = [ws, imp]
        print("Working on ws: %d, imp: %s"%(ws,imp))
        
        X_train, y_train, _ = create_train_data(window_size=ws, impute_method=imp)
        xg_model = xg_model.fit(X_train, y_train)
        
        res.append(xg_model.score(X_train, y_train))
        y_pred = xg_model.predict(X_train)
        res.extend([recall_score(y_train, y_pred), precision_score(y_train, y_pred)])
        
        X_test, y_test = predict(xg_model, 'XG', window_size=ws, impute_method=imp)
        res.append(xg_model.score(X_test, y_test))
        
        y_pred_prob = xg_model.predict_proba(X_test)
        res.append(roc_auc_score(y_test, y_pred_prob[:,1]))
        
        y_pred = xg_model.predict(X_test)
        res.extend([recall_score(y_test, y_pred), precision_score(y_test, y_pred)])
        
        xg_results.append(res)

Working on ws: 1, imp: mean
Working on ws: 1, imp: forw
Working on ws: 2, imp: mean
Working on ws: 2, imp: forw
Working on ws: 3, imp: mean
Working on ws: 3, imp: forw
Working on ws: 4, imp: mean
Working on ws: 4, imp: forw
Working on ws: 5, imp: mean
Working on ws: 5, imp: forw
Working on ws: 6, imp: mean
Working on ws: 6, imp: forw


In [394]:
xg_results = np.array(xg_results)

In [397]:
xg_results.tofile('../results/xg_results', sep='\n')

In [399]:
xg_results

array([['1', 'mean', '0.986872085629504', '0.05225311601150527',
        '0.956140350877193', '0.9852899134547912', '0.7632749368086555',
        '0.007285974499089253', '0.16'],
       ['1', 'forw', '0.9877530203476049', '0.12607861936720996',
        '0.9100346020761245', '0.9823790836083894', '0.7116494381646905',
        '0.0', '0.0'],
       ['2', 'mean', '0.9871543653385587', '0.0803921568627451',
        '0.9318181818181818', '0.9846572564506231', '0.7707686176889597',
        '0.0037243947858473', '0.047619047619047616'],
       ['2', 'forw', '0.9875966144132375', '0.11372549019607843',
        '0.9392712550607287', '0.9812151452891107', '0.7286990630535802',
        '0.0037243947858473', '0.011695906432748537'],
       ['3', 'mean', '0.9871936548791406', '0.08651911468812877',
        '0.9197860962566845', '0.9843904261280252', '0.7788484320539926',
        '0.00186219739292365', '0.02'],
       ['3', 'forw', '0.9880259623992838', '0.14738430583501005',
        '0.945161290322

# AdaBoost

In [395]:
from sklearn.ensemble import AdaBoostClassifier

ab_model = AdaBoostClassifier(n_estimators=100)

ab_results = []

for ws in range(1,7):
    for imp in ['mean', 'forw']:
        res = [ws, imp]
        print("Working on ws: %d, imp: %s"%(ws,imp))
        
        X_train, y_train, _ = create_train_data(window_size=ws, impute_method=imp)
        ab_model = ab_model.fit(X_train, y_train)
        
        res.append(ab_model.score(X_train, y_train))
        y_pred = ab_model.predict(X_train)
        res.extend([recall_score(y_train, y_pred), precision_score(y_train, y_pred)])
        
        X_test, y_test = predict(ab_model, 'AB', window_size=ws, impute_method=imp)
        res.append(ab_model.score(X_test, y_test))
        
        y_pred_prob = ab_model.predict_proba(X_test)
        res.append(roc_auc_score(y_test, y_pred_prob[:,1]))
        
        y_pred = ab_model.predict(X_test)
        res.extend([recall_score(y_test, y_pred), precision_score(y_test, y_pred)])
        
        ab_results.append(res)

ab_results = np.array(ab_results)

Working on ws: 1, imp: mean
Working on ws: 1, imp: forw
Working on ws: 2, imp: mean
Working on ws: 2, imp: forw
Working on ws: 3, imp: mean
Working on ws: 3, imp: forw
Working on ws: 4, imp: mean
Working on ws: 4, imp: forw
Working on ws: 5, imp: mean
Working on ws: 5, imp: forw
Working on ws: 6, imp: mean
Working on ws: 6, imp: forw


In [398]:
ab_results.tofile('../results/ab_results', sep='\n')

In [400]:
ab_results

array([['1', 'mean', '0.9860706337431114', '0.004314477468839885',
        '0.2571428571428571', '0.9853938716635913', '0.7433725448160046',
        '0.0018214936247723133', '0.06666666666666667'],
       ['1', 'forw', '0.9860573866044934', '0.003835091083413231',
        '0.22857142857142856', '0.9835486134573901',
        '0.6740974606665338', '0.020036429872495445',
        '0.10377358490566038'],
       ['2', 'mean', '0.9858208142826039', '0.011274509803921568',
        '0.25555555555555554', '0.9851375510312992', '0.762122784040555',
        '0.00931098696461825', '0.16666666666666666'],
       ['2', 'forw', '0.9859909100805574', '0.003431372549019608',
        '0.21212121212121213', '0.9828961763214772',
        '0.6838627539348625', '0.027932960893854747',
        '0.11194029850746269'],
       ['3', 'mean', '0.9856549350940018', '0.012575452716297788',
        '0.22123893805309736', '0.9850041358700002',
        '0.7446418219085961', '0.00931098696461825',
        '0.1428571428